<h2 id="Latent-Dirichlet-Allocation(LDA)-for-Topic-Modelling" style="text-align: center;">Latent Dirichlet Allocation(LDA) for Topic Modelling</h2>
<h2 id="
"></h2>
<h3 id="Using-the-Amazon-fine-food-reviews-dataset" style="text-align: center;">Using the Amazon fine food reviews dataset</h3>
<h5 id="Link-:-https://www.kaggle.com/snap/amazon-fine-food-reviews-" style="text-align: center;">Link : https://www.kaggle.com/snap/amazon-fine-food-reviews</h5>
<p style="text-align: center;">..........................................................................................................</p>
<h6 id="For-performing-LDA-based-topic-modelling-,--I-will-be-using-the-gensim-package-for-LDA-topic-modelling-&amp;--pyLDAvis-for-visualization-of-LDA-topic-model" style="text-align: center;">For performing LDA based topic modelling , I will be using the gensim package for LDA topic modelling &amp; pyLDAvis for visualization of LDA topic model</h6>

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Reading the data

In [2]:
review_data= pd.read_csv("Reviews.csv")

# print(review_data.head(2))

review_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
print("Length of the data is :" , len(review_data))

print('Unique Products : ' , len(review_data.groupby('ProductId')))


print('Unique Users : ', len(review_data.groupby('UserId')))


Length of the data is : 568454
Unique Products :  74258
Unique Users :  256059


### Cleaning Text

In [4]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()


In [5]:
import nltk
# nltk.download('stopwords') 

In [6]:
review_data.dropna(axis = 0, how ='any',inplace=True) 


review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 

print('-------Dataset --------')

print(review_data['Score'].value_counts())
print(len(review_data))

print('-------------------------')
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
df_sampled = df_short_reviews.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop = True)

print('No of Short reviews')
print(len(df_short_reviews))


-------Dataset --------
Score
5    363102
4     80654
1     52264
3     42638
2     29743
Name: count, dtype: int64
568401
-------------------------
No of Short reviews
373279


### Pre-Process the Text

In [8]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_sampled['Text']=df_sampled['Text'].apply(remove_stopwords)


### Lemmetization

In [9]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output


In [10]:
text_list=df_sampled['Text'].tolist()
print(text_list[1])

print('-'*60)

tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

wasnt cracklin like rocklin every full rocks pork rinds literally chipped tooth eating avoid itits worth vending machine quality
------------------------------------------------------------
['full', 'rock', 'pork', 'rind', 'tooth', 'avoid', 'itit', 'worth', 'machine', 'quality']


### Create vocabulary dictionary and document term matrix

In [11]:
dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]


### Creating the object for LDA model using gensim library


In [ ]:
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, 
                id2word=dictionary, 
                num_topics=10, 
                random_state=100,
                chunksize=1000, 
                passes=50,
                iterations=100)

In [12]:
lda_model.print_topics()

[(0,
  '0.044*"price" + 0.042*"store" + 0.032*"product" + 0.029*"great" + 0.028*"good" + 0.024*"amazon" + 0.022*"order" + 0.019*"time" + 0.018*"local" + 0.015*"year"'),
 (1,
  '0.069*"vanilla" + 0.047*"morning" + 0.030*"yummy" + 0.030*"french" + 0.029*"pill" + 0.029*"seed" + 0.023*"combination" + 0.019*"espresso" + 0.016*"salad" + 0.015*"nuts"'),
 (2,
  '0.060*"chocolate" + 0.049*"chip" + 0.042*"good" + 0.023*"taste" + 0.023*"sweet" + 0.022*"bar" + 0.021*"flavor" + 0.021*"great" + 0.019*"smooth" + 0.018*"protein"'),
 (3,
  '0.030*"time" + 0.028*"cookie" + 0.026*"good" + 0.015*"little" + 0.013*"great" + 0.013*"bread" + 0.012*"piece" + 0.011*"work" + 0.010*"thing" + 0.010*"minute"'),
 (4,
  '0.066*"product" + 0.016*"bottle" + 0.013*"great" + 0.012*"wheat" + 0.011*"fiber" + 0.011*"flour" + 0.011*"plastic" + 0.011*"free" + 0.010*"container" + 0.010*"recipe"'),
 (5,
  '0.070*"snack" + 0.047*"salt" + 0.037*"candy" + 0.037*"butter" + 0.031*"great" + 0.025*"peanut" + 0.024*"popcorn" + 0.023*"c

### Visualize the topics

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.150137  0.050384       1        1  20.176015
0     -0.142625 -0.159713       2        1  16.913468
3     -0.095810 -0.151376       3        1  11.133467
8     -0.089063 -0.051660       4        1  10.672381
4      0.009040 -0.175216       5        1   9.016446
6     -0.085106  0.055700       6        1   8.534662
2     -0.088470  0.151307       7        1   8.342582
9      0.113065  0.208595       8        1   7.017723
5      0.093137  0.202941       9        1   5.529211
1      0.435968 -0.130961      10        1   2.664044, topic_info=            Term          Freq         Total Category  logprob  loglift
71        coffee  32581.000000  32581.000000  Default  30.0000  30.0000
35          food  21949.000000  21949.000000  Default  29.0000  29.0000
96        flavor  32222.000000  32222.000000  Default  28.0000  28.0000
8        product  29251.000000  29251.000000  Default  27.0000  27.0000
9          store  13755.000000  13755.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
9795  delightful    436.885483    437.833736  Topic10  -4.7811   3.6232
708         cent    436.326133    437.274333  Topic10  -4.7823   3.6232
1028    addicted    424.605632    425.553930  Topic10  -4.8096   3.6231
309      morning   2443.819140   4155.759628  Topic10  -3.0594   3.0944
2943        fast    737.448403   1261.809564  Topic10  -4.2575   3.0882

[520 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1028     10  0.998698    addicted
1607      9  0.998567   addictive
131       4  0.999098       adult
2402      4  0.998391  affordable
2138      8  0.999530   afternoon
...     ...       ...         ...
183       3  0.155634        year
183       4  0.227962        year
183       5  0.092777        year
331       5  0.997202       yeast
2472     10  0.999207       yummy

[745 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 4, 9, 5, 7, 3, 10, 6, 2])

###  measuring how good the model is. lower the better.

In [14]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))


Perplexity:  -8.899764404994013


### Computing Coherence Score

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)

#### Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

### Method to find optimal number of topics 

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

### Visualizatoin with Graph

In [ ]:
limit=50; start=2; step=1;
x = range(start, limit, step)

plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')

# Print the coherence scores
plt.show()



### Printing the coherence scores

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))


### Selecting the model and printing the topics


In [ ]:
optimal_model = model_list[7]

model_topics = optimal_model.show_topics(formatted=False)

optimal_model.print_topics(num_words=10)

### Visualize the topics

In [ ]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(optimal_model, doc_term_matrix, dictionary)

vis

### Saving to PDF

In [ ]:
!jupyter nbconvert --to webpdf --allow-chromium-download Topic_Modeling(LDA).ipynb